# ADM Homework 2 (Group # 42)

## Team Members 

1. Syed Muhammad Hassan Raza (hasanrezacs@gmail.com)
2. Daniele Pristerà (pristera.1657095@studenti.uniroma1.it)
3. Riccardo Agabiti (riccardoagabiti98@gmail.com)

### Installing/Importing Libraries

In [9]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib.pyplot
# !pip install math
# !pip install pandasql

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from pandasql import sqldf

### Loading Dataset

As the given dataset has timestamp values in unix, we need a function to convert unix timestamps to datetime values.

In [3]:
# Below function converts unix timestamps into datetime

def dateparse(time_in_secs):
    return pd.to_datetime(time_in_secs, unit='s')

In [10]:
# Reading steam_reviews.csv file into a pandas dataframe

df = pd.read_csv('.\steam_reviews.csv', header='infer', nrows=1000000, parse_dates=['timestamp_created',
'timestamp_updated', 'author.last_played'], date_parser=dateparse)

In [ ]:
# First 5 rows of the dataset

df.head(5)

# Research Questions

## [RQ1] Exploratory Data Analysis (EDA)

In [ ]:
# Information about dataset dimension

nrows, ncolumns = df.shape
print("Dataset has {} rows and {} columns".format(nrows, ncolumns))

In [ ]:
# Information about dataset columns

print("Dataset has the following columns:")
df.columns

In [ ]:
# .info() function shows brief information about the dataset

print("Brief information about the dataset:\n")
df.info()

In [ ]:
print("The first review was created on:")
df.timestamp_created.min()

In [ ]:
print("While the latest review was updated on:")
df.timestamp_updated.max()

In [ ]:
lang_count = df.language.nunique()

print("Reviews are written in the following {} languages:".format(lang_count))
df.language.unique()

In [ ]:
print("Count of reviews per each language is:")
df.groupby('language').review_id.count().sort_values(ascending=False)

In [ ]:
print("No. of reviews wrt. day of the week:")
df.groupby(df.timestamp_created.dt.weekday).review_id.count()

In [ ]:
# Descriptive statistics wrt timestamp_created and language of the review

df.groupby([df.timestamp_created.dt.hour, df.language]).describe()

In [ ]:
# Plot of the review count wrt language in which the review is written

df['language'].value_counts().plot.bar(\
figsize = (18, 9),\
title='Top Languages', \
xlabel='Language',\
ylabel='Number of Reviews')

In [ ]:
# Piechart for the distribution of languages

plt.rcParams['figure.figsize'] = (8, 8)
plt.pie(df['language'].value_counts(),
labels = df.language.unique(),
explode = [0.1 for value in range(0, df.language.nunique())],
shadow = True, autopct = '%.1f%%')
plt.title('Languages', fontsize = 20)
plt.axis('off')
plt.show()

In [ ]:
hour_reviews = df.groupby(df.timestamp_created.dt.hour)

plt.figure()
hour_reviews.votes_funny.sum().plot()
plt.xlabel("Hour of Day")
plt.ylabel("Total Funny Votes")
plt.xticks(range(0,24))
plt.show()

There's a **sqldf** function inside pandasql library, which allows us to interact with a dataframe with SQL queries. It also returns a dataframe object. Therefore we use sqldf function for most of the questions.

## [RQ2]

### number of reviews for each application in descending order?

We use app_id column to group the number of reviews because it's a unique identity for each application.

In [ ]:
df['app_id'].value_counts().plot.bar(title='# of reviews for each application',\
                                     xlabel='app_id',\
                                     ylabel='count',
                                     figsize= (18, 9))

# plot represent the no. of reviews for each application in descending order

### What applications have the best Weighted Vote Score?

For each application, we found the max of its weighted_vote_score and sorted the result in descending order to know what applications have the best weighted score.

In [ ]:
best_weighted_score_apps = sqldf("select distinct(app_id), app_name, max(weighted_vote_score) AS Weight_Score\
                                  from df group by app_id order by Weight_Score desc")

best_weighted_score_apps

# table representing apps with the best Weighted Score (top to bottom)

### Which applications have the most and the least recommendations?

For each application, we found the review count where a review was given a positive recommendation and sorted the result in descending order to know what applications have the most recommendations.

In [ ]:
most_recommended_apps = sqldf("select distinct(app_id), app_name, count(recommended) AS Count\
                               from df where recommended=True group by app_id order by Count desc")

most_recommended_apps

# table representing most recommended apps (top to bottom)

For each application, we found the review count where a review was given a positive recommendation and sorted the result in ascending order to know what applications have the least recommendations.

In [7]:
#least_recommended_apps = sqldf("select distinct(app_id), app_name, count(recommended) AS Count\
                              # from df where recommended=True group by app_id order by Count asc")

#least_recommended_apps
least_recommended_apps = db.query("select distinct(app_id), app_name, count(recommended) AS Count\
                               from 'df.parquet' where recommended=True group by app_id order by Count asc").to_df()

least_recommended_apps



# table representing least recommended apps (top to bottom)

RuntimeError: IO Error: No files found that match the pattern "df.parquet"

### How many of these applications were purchased, and how many were given for free?

For each application, we found how many times it was purchased on Steam by the users using **steam_purchase = True** condition. Then we summed up the purchase count for each application to get the total number of purchased applications.

In [ ]:
count_purchased = sqldf("select distinct(app_id), app_name, count(steam_purchase) AS Purchased_Count\
                         from df where steam_purchase=True group by app_id order by Purchased_Count desc")

count_purchased_ = sqldf("select sum(Purchased_Count) AS Count_Purchased from count_purchased")
count_purchased_

# Total no. of applications purchased on Steam (see Count_Purchased column)

For each application, we found how many times it was got as free by the users using **received_for_free = True** condition. Then we summed up the free count for each application to get the total number of applications received for free.

In [ ]:
count_free = sqldf("select distinct(app_id), app_name, count(received_for_free) AS Free_Count\
                    from df where received_for_free=True group by app_id order by Free_Count desc")

count_free_ = sqldf("select sum(Free_Count) AS Count_Free from count_free")
count_free_

# Total no. of total applications received for free (see Count_Free column)

## [RQ3]

### What is the most common time that authors review an application?

Since we were supposed to provide the answer in H:M format, we converted datetime values in the H:M format first. Then we counted how many reviews were written for each H:M time. We then selected the time in which the maximum number of reviews were written.

In [ ]:
most_common_time = sqldf("select strftime('%H:%M', timestamp_created) AS Time, count(review_id) AS Review_Count from df\
                          group by [Time]\
                          order by Review_Count desc")

most_common_time_ = sqldf("select Time, max(Review_Count) AS 'Count of reviews' from most_common_time")
most_common_time_

# Most common time when authors review an application (see Time column)

### Create a function that receives as a parameter a list of time intervals and returns the plot the number of reviews for each of the intervals.

In [26]:
# Function that takes a list of time intervals as a parameter and returns the plot of the no. of reviews for each of the interval

def time_interval_plot(list_):   
    x = [] # list for x-axis values (time intervals) which will be populated upon the function call
    y = [] # list for y-axis values (count of reviews in the given interval) which will be populated upon the function call
    
    for start_, end_ in list_:
        # query that gives the count of reviews for each of the provided time intervals
        query = "select count(review_id) AS Count from df\
                 where strftime('%H:%M:%S', timestamp_created) between '{}' and '{}'".format(start_, end_)
        res = sqldf(query)
        y.append(res.iloc[0]['Count']) # populate the list y
        
    for start_, end_ in list_:
        x.append("{} - {}".format(start_, end_)) # populate the list x
    
    # settings for plot
    fig = plt.figure(figsize = (18, 9)) # figure size
    plt.bar(x, y) # bar plot for x, y values
    plt.xlabel("Intervals")
    plt.xticks(rotation='vertical')
    plt.ylabel("Review Count")
    plt.title("# of reviews wrt time intervals")
    return plt

### Use the function that you created in the previous literal to plot the number of reviews between the following time intervals:

In [27]:
# Intervals given in the homework

intervals = [["06:00:00", "10:59:59"], ["11:00:00", "13:59:59"], ["14:00:00", "16:59:59"], ["17:00:00", "19:59:59"],
             ["20:00:00", "23:59:59"], ["00:00:00", "02:59:59"], ["03:00:00", "05:59:59"]]

x = time_interval_plot(intervals) # Function call
x.show()

# Plot representing the no. of reviews between the provided time intervals

MemoryError: Unable to allocate 76.3 MiB for an array with shape (10000000,) and data type object

## [RQ4]

### What are the top 3 languages used to review applications?

In [ ]:
df.groupby('language').review_id.count().nlargest(3)

# top 3 languages used to review apps

### Create a function that receives as parameters both the name of a data set and a list of languages’ names and returns a data frame filtered only with the reviews written in the provided languages.

This function filters the dataframe based on the language column values. If the review language is in one of the **langs** values, it adds that review to the filtered dataframe.

In [ ]:
# Function to filter dataframe with respect to given list of languages

def language_filter(dataframe, langs):
    return dataframe[dataframe.language.isin(langs)]

### Use the function created in the previous literal to find what percentage of these reviews (associated with the top 3 languages) were voted as funny?

In [ ]:
langs = ['english', 'russian', 'schinese'] # from part 1 of RQ4

res = language_filter(df, langs) # Function call

x = round((len(res[res["votes_funny"] != 0])/len(res))*100, 2)
print("{}% of these reviews (associated with the top 3 languages) were voted as funny".format(x))

### Use the function created in the literal “a” to find what percentage of these reviews (associated with the top 3 languages) were voted as helpful?

In [ ]:
x = round((len(res[res["votes_helpful"] != 0])/len(res))*100, 2)
print("{}% of these reviews (associated with the top 3 languages) were voted as helpful".format(x))

## [RQ5]

### Plot the top 10 most popular reviewers and the number of reviews.

To get the 10 most popular reviewers, we group the result by the unique entity: author.steamid. Then we pick the maximum value of the author.num_reviews column against each author.steamid. Then we sort the result in descending order by count and pick the top 10 rows to know who are the top 10 most popular reviewers.

In [ ]:
query = "select distinct([author.steamid]), max([author.num_reviews]) AS Count from df\
        group by [author.steamid] order by Count desc LIMIT 10"

popular_reviewers = sqldf(query)

popular_reviewers.plot(x ='author.steamid', y='Count', kind = 'bar', figsize= (18, 9))

# Plot representing the top 10 most popular reviewers and the number of reviews

### What applications did the most popular author review?

We picked the most popular reviewer's id (first one) from the previous part to filter what games he has reviewed.

In [ ]:
df[df["author.steamid"] == 76561198125392509].app_name.unique() # Copy steam id of the first author and paste here

# The most popular author reviewed the following apps

### How many applications did he purchase, and how many did he get as free? Provide the number (count) and the percentage.

In [ ]:
top_reviewer = df[df["author.steamid"] == 76561198125392509] # all reviews of the most popular reviewer

#apps purchased
purchased_count = len(top_reviewer[top_reviewer["steam_purchase"] == True]) # count of purchased apps
purchased_percentage = round(purchased_count/len(top_reviewer)*100, 2) # percentage of purchased apps

#apps for free
free_count = len(top_reviewer[top_reviewer["received_for_free"] == True]) # count of apps received for free
free_percentage = round(free_count/len(top_reviewer)*100, 2) # percentage of apps received for free

print("The most popular reviewer purchased {} ({}% of the) application(s) on Steam.\n\
The most popular reviewer got {} ({}% of the) application(s) for free.".format(purchased_count, purchased_percentage, 
                                                                                       free_count, free_percentage))

In [ ]:
#top_reviewer

### How many of the applications he purchased reviewed positively, and how many negatively? How about the applications he received for free?

In [ ]:
apps_purchased = top_reviewer[top_reviewer["steam_purchase"] == True] # all reviews where apps were purchased
apps_free = top_reviewer[top_reviewer["received_for_free"] == True] # all reviews where apps were received as free

In [ ]:
#apps_purchased
#apps_free

In [ ]:
purchased_positive_count = len(apps_purchased[apps_purchased["recommended"] == True]) # count of purchased apps where they were reviewed positively
purchased_negative_count = len(apps_purchased[apps_purchased["recommended"] == False]) # count of purchased apps where they were reviewed negatively

print("Out of all the applications he purchased, {} applications reviewed positively.".format(purchased_positive_count))
print("Out of all the applications he purchased, {} applications reviewed negatively.\n".format(purchased_negative_count))

free_positive_count = len(apps_free[apps_free["recommended"] == True]) # count of free apps where they were reviewed positively
free_negative_count = len(apps_free[apps_free["recommended"] == False]) # count of free apps where they were reviewed negatively
print("Out of all the applications he received for free, {} applications reviewed positively.".format(free_positive_count))
print("Out of all the applications he received for free, {} applications reviewed negatively.".format(free_negative_count))

## [RQ6]

### What is the average time (days and minutes) a user lets pass before he updates a review?

In [ ]:
# Calculating time difference (in seconds) for each of the review in the dataset
query1 = "select review_id, timestamp_created, timestamp_updated,\
         Cast((JulianDay(timestamp_updated) - JulianDay(timestamp_created))*24*60*60 As Integer) AS DEL from df\
         where DEl <> 0 order by DEL desc"

res1 = sqldf(query1)

# Calculating average time difference (in seconds)
query2 = "select cast(AVG(DEL) as integer) AS 'Average_Time' from res1"
res2 = sqldf(query2)

# Converting time in seconds to days and minutes
query3 = "SELECT CAST((Average_Time) / (60 * 60 * 24) AS TEXT) || ' day(s) and ' ||\
         CAST(((Average_Time) % (60 * 60 * 24)) / (60) AS TEXT) || ' minute(s)' AS Avg_Time from res2"

time_ = sqldf(query3)
time_ # Average time (in days and minutes) a user lets pass before he updates a review

### Plot the top 3 authors that usually update their reviews.

In [ ]:
query = "select distinct([author.steamid]), count(review_id) AS Count from df\
         where timestamp_created <> timestamp_updated group by [author.steamid] order by Count desc LIMIT 3"

sqldf(query).plot(x ='author.steamid', y='Count', kind = 'bar') # plot representing reviews update count of top 3 authors

## [RQ7]

In [ ]:
#The probability of having a review with score bigger than 0.5 can be calculeted dividing the number of favorable cases  
# with the total number of reviews.

numb_greater_05 = sqldf("select count(*) AS Weight_Score\
            from df where weighted_vote_score > 0.5")

total_numb= sqldf("select count(*) AS Weight_Score\
            from df")
probability_greater05=numb_greater_05/total_numb
probability_greater05

The conditional probability between having a review labeled as funny given the a score gretar than 0.5 is calculated dividing the intersection between the two sets (the reviews that are simultaneously finny and with score greater than 0.5) and the numbers of review with score greater then 0.5

In [ ]:

numb_greater05_and_funny= sqldf("select count(*) AS Weight_Score\
            from df where (weighted_vote_score > 0.5) and (votes_funny >= 1)")
probability_funny_given_greater05=numb_greater05_and_funny/numb_greater_05
probability_funny_given_greater05

To verify if the two probabilities (P(score>0.5) and P(isfunny)) are indipendent we should check if they factorized, so if (P(A|B)=P(A)*P(B))

In [ ]:
#to find the probabilities of a review being funny
number_of_funny_reviews=sqldf("select count(*) AS Weight_Score\
            from df where (votes_funny >= 1)")

probability_review_funny= number_of_funny_reviews/total_numb

In [ ]:
#to calculate the total probabilities
prob_total=probability_greater05*probability_review_funny
prob_total

Since the equation (P(A|B)=P(A)*P(B)) is not verified, the two probabilities are dependent

## [RQ8]

# Theoretical Questions

## [TQ1]


# Solution of TQ1

![alt text](TQ1_1.jpg "")

## [TQ2]

![alt text](TQ2_1.jpg "")

![alt text](TQ2_3.jpg "")

![alt text](TQ2_4.jpg "")

![alt text](TQ2_5.jpg "")

![alt text](TQ2_6.jpg "")

## [TQ3]

![alt text](TQ3_1.jpg "")